# Claude + Strata Integration

This tutorial demonstrates how to build AI agents using Anthropic's Claude with Klavis Strata MCP servers for enhanced functionality.

## Prerequisites

Before we begin, you'll need:

- **Anthropic API key** - Get at [console.anthropic.com](https://console.anthropic.com/)
- **Klavis API key** - Get at [klavis.ai](https://klavis.ai/)

In [ ]:
# Install the required packages
%pip install -q klavis python-dotenv anthropic

In [ ]:
import os
import webbrowser
from klavis import Klavis
from klavis.types import McpServerName, ToolFormat
from anthropic import Anthropic

# Set environment variables
os.environ["ANTHROPIC_API_KEY"] = "YOUR_ANTHROPIC_API_KEY"  # Replace with your actual Anthropic API key
os.environ["KLAVIS_API_KEY"] = "YOUR_KLAVIS_API_KEY"  # Replace with your actual Klavis API key

## Step 1: Create Strata MCP Server

Create a unified MCP server that combines multiple services (Gmail and Slack) for enhanced agent capabilities.

In [ ]:
klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

# Create a Strata MCP server with Gmail and Slack integrations
response = klavis_client.mcp_server.create_strata_server(
    user_id="1234",
    servers=[McpServerName.GMAIL, McpServerName.SLACK],
)

print(f"🚀 Strata MCP server created successfully!")

# Handle OAuth authorization if needed
if response.oauth_urls:
    for server_name, oauth_url in response.oauth_urls.items():
        webbrowser.open(oauth_url)
        print(f"🔐 Opening OAuth authorization for {server_name}")
        input(f"Press Enter after completing {server_name} OAuth authorization...")

## Step 2: Setup Claude with MCP Tools

Set up Claude to use tools from the Strata MCP server.

In [ ]:
# Initialize Claude client
claude_client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

# Get MCP server tools in Anthropic format
mcp_server_tools = klavis_client.mcp_server.list_tools(
    server_url=response.strata_server_url,
    format=ToolFormat.ANTHROPIC
)

print(f"🤖 Claude configured with {len(mcp_server_tools.tools)} MCP tools!")

## Step 3: Create Agent Loop

Implement an agentic loop that allows Claude to use MCP tools to complete tasks.

In [ ]:
# Define user query
user_query = "Check my latest 5 emails and summarize them in a Slack message to #general channel"

messages = [
    {"role": "user", "content": user_query}
]

# Agent loop to handle tool calls
max_iterations = 10
iteration = 0

while iteration < max_iterations:
    iteration += 1
    
    claude_response = claude_client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=4000,
        system="You are a helpful assistant. Use the available tools to answer the user's question.",
        messages=messages,
        tools=mcp_server_tools.tools
    )
    
    messages.append({"role": "assistant", "content": claude_response.content})
    
    if claude_response.stop_reason == "tool_use":
        tool_results = []
        
        for content_block in claude_response.content:
            if content_block.type == "tool_use":
                function_name = content_block.name
                function_args = content_block.input
                
                print(f"🔧 Calling: {function_name}, with args: {function_args}")
                
                result = klavis_client.mcp_server.call_tools(
                    server_url=response.strata_server_url,
                    tool_name=function_name,
                    tool_args=function_args
                )
                
                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": content_block.id,
                    "content": str(result)
                })
        
        messages.append({"role": "user", "content": tool_results})
        continue
    else:
        final_response = claude_response.content[0].text
        print("\n✅ Final Response:")
        print(final_response)
        break

## Summary

🎉 Congratulations! You've successfully created a Claude agent that can:

1. **Read emails** using the Gmail MCP server
2. **Send Slack messages** using the Slack MCP server
3. **Coordinate multiple services** through Klavis Strata MCP integration

This demonstrates the power of combining Claude's advanced reasoning capabilities with Klavis MCP servers for building sophisticated AI workflows that can interact with multiple external services seamlessly.